In [1]:
import gradio as gr
import os
import time
from crewai import Agent, Task, Crew, Process
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
from crewai_tools import tool
from datetime import datetime

#from docx import Document
from io import BytesIO
import base64

import numpy as np
import pandas as pd
from IPython.display import Markdown, display

from dotenv import load_dotenv
#load_dotenv("./.env")
load_dotenv("/home/jovyan/Albert/.env")

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")
os.environ["SLIP_AUTH_URL"] = os.environ.get("SLIP_AUTH_URL")
os.environ["SLIP_USERNAME"] = os.environ.get("SLIP_USERNAME")
os.environ["SLIP_PASSWORD"] =  os.environ.get("SLIP_PASSWORD")
os.environ["SLIP_CLIENT_ID"]= os.environ.get("SLIP_CLIENT_ID")
os.environ["SLIP_APP_NAME"]=os.environ.get("SLIP_APP_NAME")
os.environ["SERPER_API_KEY"] = "25d43901c968b0f8830387b7ed72231673428b9c"

/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from llm_utils import get_llm
llm = get_llm('gpt-4o-0806',temperature=0)

In [3]:
df=pd.read_csv('/home/jovyan/data/LLM/EMR_LLM_data_combo_2204mcids_NEW.csv')

/home/jovyan/tmp/ipykernel_642872/3347160452.py:1: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/home/jovyan/data/LLM/EMR_LLM_data_combo_2204mcids_NEW.csv')


In [4]:
df.head()

,ID,DOS,TYPE,CODE,VALUE,ABNORMAL_CD,CD_DESC
0,33260,2019-10-23,EMR_Note,chief complaint,pre-op evaluation; hypertension. needs cx for ...,NaN,NaN
1,33260,2019-10-23,EMR_Note,past medical history,anemia of chronic disease. Percutaneous Corona...,NaN,NaN
2,33260,2019-11-16,EMR_Note,history of present illness,Pt is here for follow up from the hospital. Pt...,NaN,NaN
3,33260,2019-11-16,EMR_Note,physical exam,Cardiology Brief Exam. Basic Cardio PE:. HEENT...,NaN,NaN
4,33260,2019-11-16,EMR_Note,instructions,I10: Essential (primary) hypertension. 5: Hype...,NaN,NaN


In [5]:
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

In [6]:
# Define the new basic member information agent
basic_info_agent = Agent(
    role="Basic Member Information Provider",
    goal="Display the member ID, age, and gender for the specified member",
    backstory="You are a data retrieval specialist responsible for quickly and accurately providing basic member information.",
    verbose=True,
    allow_delegation=False,
    llm=llm  
)

In [7]:

diagnosis_agent = Agent(
    role="Diagnosis Analyst",
    goal="Provide a comprehensive analysis of the patient's diagnostic history",
    backstory="You are an expert in medical diagnostics with years of experience in analyzing patient histories.",
    verbose=True,
    allow_delegation=False,
    llm=llm  
)

procedure_agent = Agent(
    role="Procedure Analyst",
    goal="Analyze and summarize all medical procedures the patient has undergone",
    backstory="You are a medical procedure specialist with deep knowledge of various medical interventions.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

medication_agent = Agent(
    role="Medication Analyst",
    goal="Provide a detailed overview of the patient's medication history",
    backstory="You are a clinical pharmacologist with expertise in analyzing medication regimens and their effects.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

lab_test_agent = Agent(
    role="Lab Test Analyst",
    goal="Analyze and interpret all laboratory tests the patient has undergone",
    backstory="You are an expert in analyzing laboratory test codes and results. Your summaries are renowned for their depth, clarity, and insightful interpretation of lab test patterns and their clinical significance.",
    verbose=True,
    allow_delegation=False,
    llm=llm
)

In [8]:
# Define the prompt for basic member information retrieval
BASIC_INFO_PROMPT = """
Retrieve and display the following information for the member:
1. Member ID: {member_id}
2. Age: {age}
3. Gender: {gender}


Example :
Member Information

Member ID: 10
Age: 35
Gender: Male

Format the output like the top in markdown format.

Note: Display "Female" if the value of gender is 0, "Male" if the value of gender is 1, and "Unknown" for any other value.
"""

In [9]:
DIAG_SUMMARY_PROMPT = """
Task Description
Your task is to summarize the following 'Patient Diagnosis Code History' and and ensure that ALL diagnosis codes and their associated dates are included in the summary.

Input Data
Patient Diagnosis Code History:
{diagnosis_history}

Output Format
Generate the summary in the following markdown format:

Diagnosis Summary:

Diagnosis Category 1:
- Diagnosis 2 (code2) in [dos1, dos2, dos3]
- Diagnosis 1 (code1) in [dos1, dos2, dos3, dos4, dos5]

Diagnosis Category 2:
- Diagnosis 6 (code6) in [dos1, dos2, dos3]
- Diagnosis 4 (code4) in [dos1]

Diagnosis Category 3: 
- Diagnosis 3 (code3) in [dos1, dos2, dos3, dos4, dos5]
- Diagnosis 5 (code5) in [dos1, dos2, dos3, dos4, dos5, dos6]

Rules and Guidelines

1. Include **all diagnosis codes and their associated dates** explicitly present in the provided input data. **Do not exclude any diagnosis code or date**. 
2. Parse the input systematically to ensure every diagnosis code is processed and included in the summary. 
3. Each diagnosis should appear only once, with all associated dates listed chronologically. 
4. Prioritize more serious and then recent diagnosis categories to the top. 
5. Within each category, prioritize more serious and recent diagnosis to the top. 
6.If a diagnosis code does not fit into a designated category, list it as a stand-alone item.
7.Before finalizing, verify that no diagnosis codes or dates are missing by comparing the summary with the input data. 

Special Case
If there are no diagnoses in the patient's history, output only:
"There are no diagnoses observed in the patient's history."

Diagnosis Summary:
""".strip()

In [10]:
PROCEDURE_PROMPT = """

Task Description
Your task is to summarize the following 'Patient Procedure Code History' and ensure that ALL procedures and their associated dates are included in the summary.

Input Data
Patient Procedure Code History:
{procedure_history}

Output Format
Generate the summary in the following markdown format:

Procedure Summary:

Procedure Category 1:
- Proc 2 (code2) in [dos1, dos2, dos3]
- Proc 1 (code1) in [dos1, dos2, dos3, dos4, dos5]

Procedure Category 2:
- Proc 6 (code6) in [dos1, dos2, dos3, dos4, dos5]
- Proc 4 (code4) in [dos1, dos2, dos3]

Procedure Category 3: 
- Proc 3 (code3) in [dos1, dos2, dos3]
- Proc 5 (code5) in [dos1, dos2]

Rules and Guidelines

1. Include **all procedures and their associated dates** explicitly present in the provided input data. **Do not exclude any procedure or date**. 
2. Parse the input systematically to ensure every procedure is processed and included in the summary. 
3. Each procedure should appear only once, with all associated dates listed chronologically. 
4. Prioritize more serious and then recent procedure categories to the top. 
5. Within each category, prioritize more serious and recent procedures to the top. 
6.If a procedure does not fit into a designated category, list it as a stand-alone item.
7.Before finalizing, verify that no procedure codes or dates are missing by comparing the summary with the input data. 

Special Case
If there are no procedures in the patient's history, output only:
"There were no procedures performed in the patient's history."

Procedure Summary:
""".strip()


In [11]:
MEDICATION_PROMPT = """
Task Description
Your task is to summarize the following 'Patient RX Code History' and ensure that ALL medications and their associated dates are included in the summary.

Input Data
Patient RX Code History:
{medication_history}

Output Format
Generate the summary in the following markdown format:

Prescription Medication Summary:

Medication Category 1:
- Rx 2 (code2) in [dos1, dos2, dos3]
- Rx 1 (code1) in [dos1, dos2, dos3, dos4]

Medication Category 2:
- Rx 6 (code6) in [dos1, dos2, dos3, dos4, dos5]
- Rx 4 (code4) in [dos1, dos2, dos3]

Medication Category 3: 
- Rx 3 (code3) in [dos1, dos2, dos3, dos4, dos5]
- Rx 5 (code5) in [dos1, dos2, dos3, dos4, dos5, dos6]

Rules and Guidelines

1. Include **all medications and their associated dates** explicitly present in the provided input data. **Do not exclude any medication or date**. 
2. Parse the input systematically to ensure every medication is processed and included in the summary. 
3. Each medication should appear only once, with all associated dates listed chronologically. 
4. Prioritize more serious and then recent medications categories to the top. 
5. Within each category, prioritize more serious and recent procedures to the top. 
6.If a medication does not fit into a designated category, list it as a stand-alone item.
7.Before finalizing, verify that no medication codes or dates are missing by comparing the summary with the input data. 


Special Case
If there are no medications in the patient's history, output only:
"There were no medications prescribed in the patient's history."


Prescription Medication Summary :
""".strip()


In [12]:
LAB_TEST_PROMPT ="""
Task Description
Your task is to summarize the following 'Patient Lab Code History' and ensure that ALL lab tests and their exact abnormal code values from the input data are included in the summary.

Input Data
Patient Lab Code History:
{lab_test_history}

Output Format
Generate the summary in the following markdown format:
Lab Summary:

Lab Category 1:
- Complete Lab Test Name (code) : [date1:value1 abnormal_code_value, date2:value2 abnormal_code_value]

Lab Category 2:
- Complete Lab Test Name (code) : [date1:value1 abnormal_code_value, date2:value2 abnormal_code_value]

Rules and Guidelines
1.Include **ALL lab tests, dates, values, and exact abnormal code values** explicitly present in the input data. **Do not exclude any lab test or associated details.**
2. Parse the input systematically to ensure every lab test is processed and included in the summary. 
3. Consolidate all dates and results for each lab test, listing them in chronological order within square brackets. 
4. If a value or abnormal code is missing, leave it blank
5. Use the exact abnormal code values as provided in the input, without any transformation or mapping. 
6. Categorize lab tests into appropriate groups based on the test type. 
7. Tests without a clear category should be listed as stand-alone items. 
8. Prioritize more serious and then recent lab categories at the top. 
9. Within each category, prioritize more serious and recent lab tests at the top
10. Each lab test should appear only once in the summary, without duplication, consolidating all associated details. 
11.**Before finalizing the output, verify that all lab tests and their details from the input are included in the summary.**

Special Case
If there are no lab tests in the patient's history, output only:
"There were no lab tests conducted in the patient's history."

Lab Summary:
""".strip()

In [13]:
# Tasks for each agent
# Create a task for basic member information retrieval
basic_info_task = Task(
    description=BASIC_INFO_PROMPT,
    agent=basic_info_agent,
    expected_output="A concise display of the member's ID, age, and gender."
)

diagnosis_task = Task(
    description=DIAG_SUMMARY_PROMPT,
    agent=diagnosis_agent,
    expected_output="A comprehensive analysis of the patient's diagnostic history, or a statement indicating no diagnoses if applicable"
)

procedure_task = Task(
    description=PROCEDURE_PROMPT,
    agent=procedure_agent,
    expected_output="A detailed summary of all medical procedures the patient has undergone, or a statement indicating no procedures if applicable"
)

medication_task = Task(
    description=MEDICATION_PROMPT,
    agent=medication_agent,
    expected_output="A comprehensive summary of the patient's medication history, or a statement indicating no medications if applicable"
)

lab_test_task = Task(
    description=LAB_TEST_PROMPT,
    agent=lab_test_agent,
    expected_output="A detailed analysis of all lab tests with results the patient has undergone, or a statement indicating no lab tests if applicable"
)


In [14]:
past_condition_agent = Agent(
    role="Past Medical Condition Analysis Agent", 
    goal=("Thoroughly analyze patient medical history to identify the most serious past medical conditions. "
          "List up to 10 conditions, ordered by severity, with the most serious condition first. "
          "Provide detailed and complete evidence for each condition, including all relevant medical codes, dates, and context, "
          "ensuring no evidence is omitted if it is present in the original data. " 
          "For lab test results, focus on abnormal values (e.g., H, L) and exclude those considered normal (e.g., UNK). "
          "Include the abnormal code alongside the values. " 
          "For each abnormal lab test result, provide the typical normal reference range to allow for comprehensive comparison. "
          "If evidence exists for a particular medical code type (e.g., diagnosis, procedure, lab tests, or medications), "
          "it must be included in the output without omission. " 
          "Use the complete name of medical conditions, avoiding short forms or abbreviations." ), 
    verbose=True,
    allow_delegation=False,
    llm=llm,
    backstory=("As a medical analyst, you specialize in comprehensively examining patient medical history to identify the " 
               "most serious past medical conditions. Your role involves listing conditions, prioritizing them by severity, and "
               "providing detailed evidence. Your output should be professional, thorough, and structured. "
    )
)

past_condition_task = Task(
    description=( "Analyze the given {patient_medical_history} to identify the most serious past medical conditions.\n" 
                 "1. Generate a **list of up to 10 of the most serious past medical conditions, prioritizing them based on severity, frequency, " 
                 "and long-term impact**.\n" 
                 "2. For each condition, provide detailed evidence, including:\n" 
                 " - Diagnostic Evidence:\n" 
                 " - Include all diagnosis codes with dates and context. Ensure no relevant diagnosis evidence is missed.\n" 
                 " - Procedural Evidence:\n" 
                 " - Include all relevant procedure codes linked to the condition, with dates and context. Ensure completeness.\n"
                 " - Lab Test Results:\n" 
                 " - Include all relevant lab tests the patient has undergone, with lab test codes, dates, values, abnormal codes, and context.\n"
                 " - For each abnormal result (e.g., H, L), provide the typical normal reference range for comparison.\n"
                 " - Provide a detailed explanation of how the test results deviate from normal ranges and support the condition.\n"
                 " - Ensure every abnormal result is highlighted and linked to the condition.\n"
                 " - Medication History:\n" 
                 " - Include all medication codes prescribed that relate to the condition, with dates and context. Ensure completeness.\n" 
                 "3. If evidence exists for a particular medical code type (e.g., procedures, lab tests, or medications), it must be included in the output.\n"
                 "4. If no evidence is found for a particular medical code type (e.g., procedures, lab tests, or medications), "
                "omit the corresponding evidence section for that condition.\n"
                 "5. Clearly explain for each condition:\n" 
                 " - Why this condition is identified as a past medical condition.\n" 
                 " - How the available evidence supports this condition.\n" ), 
    expected_output=( "A comprehensive markdown-formatted analysis that includes:\n"
                     "# Comprehensive Past Medical Condition Analysis\n\n" 
                     "## List of Top 10 Most Serious Past Medical Conditions\n\n" 
                     "## Detailed Analysis for Each Condition\n\n"
                     " - Diagnostic Evidence:\n"
                     " - All diagnosis codes: Code, date, and context.\n" 
                     " - Procedural Evidence:\n"
                     " - All relevant procedures, including codes, dates, and context.\n"
                     " - Lab Test Results:\n"
                     " - Relevant lab tests, including:\n"
                     "   * Lab test codes\n"
                     "   * Dates\n"
                     "   * Actual values\n"
                     "   * Abnormal codes\n"
                     "   * Typical normal reference range\n"
                     "   * Context and explanation of how results deviate from normal ranges\n" 
                     " - Medication History:\n"
                     " - Relevant medications prescribed for the condition, including medication codes, dates, and context.\n" 
                     " - Explanation:\n"
                     " - How the evidence collectively indicates the condition.\n"
                     "If no evidence exists for a particular medical code type, omit the corresponding evidence section for that condition."), 
    agent=past_condition_agent)

In [15]:
future_condition_agent = Agent( role="Future Medical Condition Predictor",
                               goal=( "Analyze the patient's medical history and predict potential future medical conditions. "
                                     "For each predicted condition, provide detailed, evidence-backed explanations using diagnostic, procedural, lab test, and medication data. "
                                     "Ensure evidence is aligned with past medical condition results and highlights abnormalities clearly. " 
                                     "Include the estimated time frame, primary and secondary risk factors, and recommended preventive measures. " 
                                     "Always include lab test codes, values, and trends explicitly in the evidence. Use the complete name of medical conditions and avoid abbreviations." ), 
                               tools=[scrape_tool, search_tool], 
                               verbose=True, 
                               llm=llm,
                               backstory=( "As a Predictive Medical Analyst, you specialize in leveraging diagnostic codes, procedures, lab test results, "
                                          "and medication history to predict potential future medical conditions. " 
                                          "Your work prioritizes evidence-backed predictions that are aligned with past medical condition insights and proactive care strategies, " 
                                          "with an emphasis on clarity, accuracy, and actionable recommendations." ) )

future_condition_task = Task( description=( "Analyze the given {patient_medical_history} to predict potential future medical conditions:\n\n" 
                                           "1. Generate a **complete list of potential future medical conditions**.\n\n" 
                                           "2. For each predicted condition, provide detailed evidence and recommendations, including:\n" 
                                           " - **Diagnostic Evidence**:\n" 
                                           " - Relevant past diagnosis codes and their connections to the prediction.\n" 
                                           " - Explanation of how the diagnostic history supports the prediction.\n"
                                           " - Ensure this section strictly contains lab-related evidence, avoiding misclassification.\n\n"
                                           " - **Procedural Evidence**:\n" 
                                           " - Relevant Past procedures codes and their connections to the predicted condition.\n"
                                           " - Explanation of their role in the prediction.\n" 
                                           " - Ensure this section strictly contains lab-related evidence, avoiding misclassification.\n\n"
                                           " - **Lab Test Evidence**:\n" 
                                           " - Include specific lab test codes, values, and trends (e.g., persistent low GFR, high creatinine levels).\n" 
                                           " - Focus on abnormal evidence only, ignoring normal or unknown (UNK) values unless critical.\n" 
                                           " - Explanation of how lab trends or abnormalities suggest the condition.\n"
                                           " - Ensure this section strictly contains lab-related evidence, avoiding misclassification.\n\n"
                                           " - **Medication Evidence**:\n"
                                           " - Relevant Medications codes related to the predicted condition.\n" 
                                           " - Explanation of their contribution to the prediction or risk.\n\n" 
                                           " - **Estimated Timeframe**:\n"
                                           " - Approximate timeline for when the condition might occur (e.g., months, years).\n\n" 
                                           " - **Risk Factors**:\n" 
                                           " - **Primary Risk Factors**: Direct contributors (e.g., medical history, lifestyle, genetics).\n" 
                                           " - **Secondary Risk Factors**: Amplifiers or indirect contributors of risk (e.g., environmental, social).\n\n" 
                                           " - **Preventive Measures**:\n"
                                           " - Recommended actions to mitigate risks and prevent the condition.\n"
                                           " - Suggestions for monitoring and follow-ups.\n\n"
                                           "3. Validate evidence classification:\n" 
                                           " - Cross-check that diagnostic, procedural, lab test, and medication evidence are categorized accurately.\n"
                                           " - Avoid overlapping evidence unless explicitly justified.\n\n"
                                           "4. If no evidence is found for a particular medical code type (e.g., procedures, lab tests, or medications), "
                                           "omit the corresponding evidence section for that condition.\n" 
                                           "5. Clearly explain for each condition:\n" 
                                           " - Why this condition is identified as a future medical condition.\n" 
                                           " - How the available evidence supports this condition, with emphasis on accuracy and alignment." ), 
                             expected_output=( "A comprehensive markdown-formatted report including:\n\n"
                                              "# Comprehensive Future Medical Condition Analysis\n\n" 
                                              "## Prioritized List of Potential Future Medical Conditions\n\n" 
                                              "## Detailed Evidence for Each Condition\n" 
                                              " - **Diagnostic Evidence**: Past diagnosis codes and their relevance.\n" 
                                              " - **Procedural Evidence**: Past procedures codes and their connection to the prediction.\n" 
                                              " - **Lab Test Evidence**: Relevant lab test codes, values, and trends, with abnormal results highlighted.\n" 
                                              " - **Medication Evidence**: Pasr Medications codes and their relevance contributing to the prediction.\n"
                                              " - **Estimated Timeframe**: Predicted time of occurrence.\n" 
                                              " - **Risk Factors**:\n" " - Primary: Direct contributors.\n" 
                                              " - Secondary: Risk amplifiers.\n"
                                              " - **Preventive Measures**: Clear recommendations for reducing risks.\n\n" 
                                              " - Explanation:\n" 
                                              " - How the evidence collectively indicates the condition.\n\n"
                                              "If no evidence is found for a particular medical code type, omit the corresponding section in the output.\n" 
                                              "Ensure all evidence is structured, clearly explained, and aligned with past medical condition results." ), 
                                             agent=future_condition_agent, 
                                             context=[past_condition_task] )

In [16]:
question_answer_generator_agent = Agent(
    role="Medical Question and Answer Generator",
    goal="Generate three important questions with answers to help patients discuss their condition with doctors",
    tools=[search_tool, scrape_tool],
    verbose=True,
    llm=llm,
    backstory=(
        "You are an experienced medical professional with expertise in patient education and doctor-patient communication. "
        "Your primary role is to generate insightful questions and provide detailed, informative answers. This will help "
        "patients have more productive discussions with their doctors regarding their condition, treatment options, and overall health management."
    )
)

question_answer_generator_task = Task(
    description=(
        "Based on the given {patient_medical_history} medical information, generate three critical and actionable questions with detailed answers to "
        "assist the patient in discussing their health condition effectively with their doctor. Follow these steps meticulously:\n\n"
        "1. **Analyze Patient Data**: Carefully review the patient's diagnosis history, lab results, medication history, procedure history, "
        "past medical conditions, and potential risk for future conditions.\n\n"
        "2. **Identify Key Concerns**: Pinpoint the most critical aspects of the patient's health status and medical history. These concerns "
        "should form the basis for the questions.\n\n"
        "3. **Formulate Questions**: Create three specific, targeted questions that address:\n"
        "   - **Understanding the Current Condition**: Help the patient grasp their diagnosis and its implications.\n"
        "   - **Treatment or Management Options**: Explore potential treatment strategies or management approaches.\n"
        "   - **Prevention or Lifestyle Modifications**: Recommend preventive measures or lifestyle changes to improve outcomes.\n\n"
        "4. **Provide Answers**: For each question, generate a detailed, clear, and accurate answer that considers the patient's "
        "specific case and general medical knowledge. The answers should be concise yet comprehensive, offering actionable insights.\n\n"
        "5. **Explain Importance**: For each question-answer pair, include a brief explanation of why this information is vital for the patient "
        "to discuss with their doctor.\n\n"
        "6. **Output in Markdown**: The output **must** follow a markdown format with the following structure:\n"
        "   - **Title**: '# Important Questions to Discuss with Your Doctor'\n"
        "   - **Disclaimer**: Add a note indicating that the content is generated with the assistance of an LLM and are intended to help guide your discussion with your healthcare provider. They are not a substitute for professional medical advice.\n"
        "   - **Sections for Each Question-Answer Pair**: Each section should include:\n"
        "     - Question: Bolded and clearly formatted.\n"
        "     - Answer: A concise and informative response.\n"
        "     - Importance: A brief paragraph explaining why this question-answer is critical."
    ),
    expected_output=(
        "A mandatory markdown-formatted document that includes:\n\n"
        "1. **Title**: '# Important Questions to Discuss with Your Doctor'\n"
        "2. **Disclaimer Note**: Clearly states the content is LLM-generated and supplementary to professional advice.\n"
        "3. **Three Detailed Question-Answer Pairs**: Each pair includes:\n"
        "   - The question (bolded).\n"
        "   - The answer (clear, informative, and actionable).\n"
        "   - The importance (why it is essential for the patient).\n\n"
        "The document must be easy to read and tailored to the patient's medical profile, ensuring maximum value during a doctor consultation."
    ),
    agent=question_answer_generator_agent,
    context=[ 
        past_condition_task, 
        future_condition_task
    ]
)

In [17]:
import matplotlib.pyplot as plt
import io
from PIL import Image
from typing import List, Dict, Union

import os
import matplotlib.pyplot as plt
from datetime import datetime
from typing import List, Dict, Union

@tool
def plot_lab_test(test_name: str, data: List[Dict[str, Union[str, float]]], config: dict = None):
    """
    Generate a graph of lab test results over time and return a filename.
    
    Parameters:
    test_name (str): The name of the lab test, used for the graph title and y-axis label.
    data (List[Dict[str, Union[str,str]]]): A list of dictionaries, where each dictionary
        contains 'date' (str) and 'value' (float) keys representing a single test result.
    config (dict, optional): Configuration options for plot customization.
    
    Returns:
    str: Filename of the saved plot
    """
    # Set default configuration
    default_config = {
        'figsize': (10, 6),
        'marker': 'o',
        'rotation': 45,
        'color': 'blue',
        'line_style': '-'
    }
    
    # Override default config with user-provided config
    if config is not None:
        default_config.update(config)
    
    # Extract dates and values
    dates = [point['date'] for point in data]
    values = [point['value'] for point in data]
    
    # Create the plot
    plt.figure(figsize=default_config['figsize'])
    plt.plot(
        dates, 
        values, 
        marker=default_config['marker'], 
        color=default_config['color'],
        linestyle=default_config['line_style']
    )
    plt.title(f'{test_name} Test Results Over Time')
    plt.xlabel('Date')
    plt.ylabel(f'{test_name} Value')
    plt.xticks(rotation=default_config['rotation'])
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.tight_layout()
    
    # Create a directory to save plots if it doesn't exist
    os.makedirs('saved_plots', exist_ok=True)
    
    # Generate a unique filename using the test name and current timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = os.path.join('saved_plots', f"{test_name.replace(' ', '_')}_{timestamp}.png")
    
    # Save the plot as an image file
    plt.savefig(filename)
    
    # Clear the current figure to free up memory
    plt.clf()
    plt.close()
    
    return filename

In [18]:
# Define the Intelligent Medical Chatbot Agent
intelligent_medical_chatbot = Agent(
    role="Smart Medical Expert",
    goal="Provide accurate and helpful responses to user questions about medical conditions, interpret medical codes, and generate visualizations of lab test results when requested",
    tools=[search_tool, scrape_tool, plot_lab_test],  
    llm=llm, 
    backstory=(
        "You are an AI-powered medical expert chatbot with extensive knowledge of medical codes and the ability to analyze and interpret medical records. "
        "You can answer both patient-specific and general medical queries, leveraging your access to patient information including basic details, diagnosis history, "
        "procedure history, medication history, lab test results, past medical conditions, and potential future conditions. "
        "You are allowed to make multiple calls (either together or in sequence). "
        "You're equipped with a lab visualization tool for generating graphs of lab test results. "
        "Your responses are based on your comprehensive medical knowledge and the patient data provided."
    )
)

intelligent_medical_chatbot_task = Task(
    description=(
        "As a smart medical expert, your task is to answer '{user_question}' accurately and helpfully. Follow these guidelines:\n"
        "1. Analyze the user's question to determine the type of information needed.\n"
        "2. For patient-specific questions:\n"
        "   - Review and use the provided patient data.\n"
        "   - Use patient-specific information to craft a precise response.\n"
        "3. For general medical queries or questions not related to a specific patient:\n"
        "   - Utilize the search_tool or scrape_tool to gather relevant and up-to-date medical information.\n"
        "   - Cross-reference multiple sources to ensure accuracy.\n"
        "   - Synthesize the gathered information into a comprehensive response.\n"
        "4. Use your extensive medical knowledge to interpret and explain medical information.\n"
        "5. Interpret medical codes using your expertise when required.\n"
        "6. For lab test visualization:\n"
        "   - If explicitly requested or if it would enhance your explanation, use the plot_lab_test tool.\n"
        "   - Provide the exact lab_code name when calling the tool.\n"
        "   - The tool will return a filename (e.g., 'saved_plots/lab_code_timestamp.png').\n"
        "   - Include this filename in your response text.\n"
        "   - Do not attempt to embed the image yourself; the chat function will handle this.\n"
        "7. You can use the plot_lab_test tool multiple times if needed for different lab tests.\n"
        "8. Format your response clearly:\n"
        "   - Provide a direct answer to the question.\n"
        "   - Include relevant details from patient information, web search, or your medical knowledge.\n"
        "   - If you've generated a graph, mention the filename in your text response.\n"
        "   - Provide a brief interpretation of the graph.\n"
        "9. If the question is outside your scope or requires immediate medical attention, advise consulting a healthcare professional.\n"
        "10. Maintain patient confidentiality at all times.\n"
    ),
    agent=intelligent_medical_chatbot,
    context=[basic_info_task, diagnosis_task, procedure_task, medication_task, lab_test_task, past_condition_task, future_condition_task],
    expected_output=(
        "A clear, informative, and accurate response addressing the user's question. "
        "For patient-specific queries, reference the sources of patient information. "
        "For general queries, include sources from web searches. "
        "If a graph was generated, the filename should be mentioned in the text response. "
        "The response should combine textual explanation with references to generated graphs when appropriate."
    )
)

In [19]:
df.columns

Index(['ID', 'DOS', 'TYPE', 'CODE', 'VALUE', 'ABNORMAL_CD', 'CD_DESC'], dtype='object')

In [20]:
def process_member_data(member_id):
    df.columns=df.columns.str.lower()
    df_member = df[df['id'] == member_id]
   
    if df_member.empty:
        return None, None, None, None
    
    
    #extract basic information 
    df_age_gender=df_member[(df_member['type']=='age') | (df_member['type']=='gender')].reset_index(drop=True)
    age=df_age_gender.loc[0,'value']
    gender=df_age_gender.loc[1,'value']
    
    
    df_diag = df_member[df_member['type'] == 'diag']
    df_diag = df_diag[['id', 'dos', 'code', 'cd_desc', 'type']]
    df_proc = df_member[df_member['type'] == 'proc']
    df_proc = df_proc[['id', 'dos', 'code', 'cd_desc', 'type']]
    df_rx = df_member[df_member['type'] == 'gpi']
    df_rx = df_rx[['id', 'dos', 'code', 'cd_desc', 'type']]
    df_lab = df_member[df_member['type'] == 'lab']
    df_lab['value'] = pd.to_numeric(df_lab['value'], errors='coerce')
    df_lab['value']=np.round(df_lab['value'],2)
    df_lab=df_lab[['id','dos','code','value','abnormal_cd','cd_desc','type']]
    
    data =df_member[~df_member['type'].isin(['EMR_Note'])]
    data['value'] = pd.to_numeric(data['value'], errors='coerce')
    data['value'] = np.round(data['value'],2)
    
    
    basic_info_crew = Crew(agents=[basic_info_agent], tasks=[basic_info_task], verbose=True)
    basic_info_result = basic_info_crew.kickoff({"member_id": member_id,"age":age,"gender":gender})
    
#     diagnosis_crew = Crew(agents=[diagnosis_agent], tasks=[diagnosis_task], verbose=True)
#     diag_result = diagnosis_crew.kickoff({"diagnosis_history": df_diag.to_string()})
    
#     procedure_crew = Crew(agents=[procedure_agent], tasks=[procedure_task], verbose=True)
#     proc_result = procedure_crew.kickoff({"procedure_history": df_proc.to_string()})
    
#     medication_crew = Crew(agents=[medication_agent], tasks=[medication_task], verbose=True)
#     med_result = medication_crew.kickoff({"medication_history": df_rx.to_string()})
    
#     lab_test_crew = Crew(agents=[lab_test_agent], tasks=[lab_test_task], verbose=True)
#     lab_result = lab_test_crew.kickoff({"lab_test_history": df_lab.to_string()})
    
    past_condition_crew = Crew(agents=[past_condition_agent], tasks=[past_condition_task], verbose=True)
    past_condition_result = past_condition_crew.kickoff({"patient_medical_history" : data.to_string()})
    
    future_condition_crew = Crew(agents=[future_condition_agent], tasks=[future_condition_task], verbose=True)
    future_condition_result = future_condition_crew.kickoff({"patient_medical_history" : data.to_string()})
    
    question_answer_generator_crew = Crew(agents=[question_answer_generator_agent], tasks=[question_answer_generator_task], verbose=True)
    question_answer_generator_result = question_answer_generator_crew.kickoff({"patient_medical_history" : data.to_string()})
    
    
    return basic_info_result,past_condition_result,future_condition_result,question_answer_generator_result

In [21]:
def gradio_interface(member_id): 
    try:
        member_id = int(member_id) 
    except ValueError:
        return "Invalid member ID. Please enter a number.", "", "", "", "", "", "", "", ""
    
    # Process the member data and get results from other agents 
    results = process_member_data(member_id) 
    if results[0] is None:
        return "No data found for this member ID.", "", "", "", "", "", "", "", ""
    
    return results

In [22]:
import re
import base64
from PIL import Image
import io

def chat(message, history):
    med_chatbot_crew = Crew(agents=[intelligent_medical_chatbot], tasks=[intelligent_medical_chatbot_task], verbose=True)
    
    response = med_chatbot_crew.kickoff({"user_question": message, "History": history})
    
    # Use regex to find all .png filenames in the saved_plots directory
    image_paths = re.findall(r'saved_plots/[^()]+\([^)]+\)_\d+_\d+\.png', response)
    
    if image_paths:
        # Get the last image path (in case there are multiple)
        print(image_paths)
        image_path = image_paths[-1]
        print(f"Image path: {image_path}")
        
        # Extract the text response by removing the image path
        #text_response = response.replace(image_path, '').strip()
        
        try:
            with Image.open(image_path) as img:
                buffer = io.BytesIO()
                img.save(buffer, format="PNG")
                encoded_image = base64.b64encode(buffer.getvalue()).decode()
            
            full_response = f"{response}\n\nHere's the requested graph:\n\n![Lab Test Graph](data:image/png;base64,{encoded_image})"
            
            history.append((message, full_response))
        except Exception as e:
            error_response = f"An error occurred while processing the graph: {str(e)}"
            history.append((message, f"{response}\n\n{error_response}"))
    else:
        history.append((message, response))
    
    return "", history

In [23]:

def save_results(member_id, basic_info, diag, proc, med, lab, past_condition, future_condition, qa):
    results = {
        "Member ID": member_id,
        "Basic Info": basic_info,
        "Diagnosis Summary": diag,
        "Procedure Summary": proc,
        "Medication Summary": med,
        "Lab Summary": lab,
        "Past Medical Conditions": past_condition,
        "Potential Future Conditions": future_condition,
        "Common Q&A": qa
    }
    
    filename = f"results_for_patient_ID{member_id}.md"
    with open(filename, 'w') as f:
        f.write(f"# Medical Record Results for Member ID: {member_id}\n\n")
        for key, value in results.items():
            f.write(f"## {key}\n\n{value}\n\n")
    
    return f"Results saved to {filename}"

In [24]:
import gradio as gr

css = """
h1 {
    text-align: center;
    display:block;
}
"""

def create_patient_interface(num_patients):
    with gr.Blocks(fill_height=True, fill_width=True, css=css, theme=gr.themes.Soft(primary_hue=gr.themes.colors.blue, secondary_hue=gr.themes.colors.sky)) as demo:
        gr.Markdown("# Chat With Medical Records")
        gr.Markdown("Enter a member ID to get a detailed analysis of the patient's medical history.")
        
        with gr.Column(scale=6):
            with gr.Tabs() as main_tabs:
                # Lists to store components
                member_id_inputs = []
                submit_buttons = []
                basic_info_outputs = []
                diag_outputs = []
                proc_outputs = []
                med_outputs = []
                lab_outputs = []
                past_condition_outputs = []
                future_condition_outputs = []
                question_answer_outputs = []
                save_buttons = []
                save_outputs = []
               
                # Create a tab for each patient
                for i in range(1, num_patients + 1):
                    with gr.Tab(f'Patient {i}'):
                        with gr.Column():
                            # Member ID Input and Submit
                            with gr.Row():
                                member_id_input = gr.Textbox(label=f"Enter Member ID for Patient {i}")
                                submit_button = gr.Button("Submit")
                                member_id_inputs.append(member_id_input)
                                submit_buttons.append(submit_button)

                            # Results Tabs
                            with gr.Tabs() as result_tab:
                                with gr.TabItem("Member Info"):
                                    basic_info_output = gr.Markdown()
                                    basic_info_outputs.append(basic_info_output)

#                                 with gr.TabItem("Diagnosis Summary"):
#                                     #diag_output = gr.Markdown()
#                                     diag_output="NA"
#                                     diag_outputs.append(diag_output)

#                                 with gr.TabItem("Procedure Summary"):
#                                     #proc_output = gr.Markdown()
#                                     proc_output="NA" 
#                                     proc_outputs.append(proc_output)

#                                 with gr.TabItem("Medication Summary"):
#                                     med_output = gr.Markdown()
#                                     med_outputs.append(med_output)

#                                 with gr.TabItem("Lab Summary"):
#                                     lab_output = gr.Markdown()
#                                     lab_outputs.append(lab_output)

                                with gr.TabItem("Past Medical Conditions"):
                                    past_condition_output = gr.Markdown()
                                    past_condition_outputs.append(past_condition_output)

                                with gr.TabItem("Potential Future Conditions"):
                                    future_condition_output = gr.Markdown()
                                    future_condition_outputs.append(future_condition_output)

                                with gr.TabItem("Ask doctor"):
                                    question_answer_output = gr.Markdown()
                                    question_answer_outputs.append(question_answer_output)
                                


                            # Save Results
                            with gr.Row():
                                save_button = gr.Button("Save Results")
                                save_output = gr.Markdown()
                                save_buttons.append(save_button)
                                save_outputs.append(save_output)

            # Medical Chatbot Section - Outside of Patient Tabs
            #gr.Markdown("## Medical Chatbot")
            with gr.TabItem("Medical Chatbot"):
                chatbot = gr.Chatbot(label="Medical Chatbot", height=600)
                msg = gr.Textbox(label="Ask a question")
                clear = gr.Button("Clear Chat")

            # Event Handlers
            for i in range(num_patients):
                print(diag_outputs)
                # Submit button event
                submit_buttons[i].click(
                    fn=gradio_interface,  
                    inputs=member_id_inputs[i],
                    outputs=[
                        basic_info_outputs[i], past_condition_outputs[i], 
                        future_condition_outputs[i], 
                        question_answer_outputs[i]
                    ]
                )

                # Save button event
                save_buttons[i].click(
                    fn=save_results, 
                    inputs=[
                        member_id_inputs[i], 
                        basic_info_outputs[i],  past_condition_outputs[i], 
                        future_condition_outputs[i], 
                        question_answer_outputs[i]
                    ],
                    outputs=save_outputs[i]
                )

            # Chatbot events
            msg.submit(
                fn=chat,  
                inputs=[msg, chatbot], 
                outputs=[msg, chatbot]
            )

            clear.click(
                lambda: None, 
                None, 
                chatbot, 
                queue=False
            )

        return demo

# Create interface with 3 patients
demo = create_patient_interface(3)
demo.launch(share=True)

/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/gradio/components/chatbot.py:223: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(
/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/gradio/utils.py:980: UserWarning: Expected 9 arguments for function <function save_results at 0x7f06e97e8ea0>, received 5.
  warnings.warn(
/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/gradio/utils.py:984: UserWarning: Expected at least 9 arguments for function <function save_results at 0x7f06e97e8ea0>, received 5.
  warnings.warn(
which: no node in (/home/jovyan/envs/llm_agents/bin:/miniconda3/condabin:/opt/mssql-tools/bin:/usr/local/apache-jmeter-5.4.3/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/miniconda3/bin:/usr/lib/spa

[]
[]
[]
* Running on local URL:  http://127.0.0.1:7861
* Running on public URL: https://31f3c3cae4b4d29ac4.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/home/jovyan/tmp/ipykernel_642872/1782168925.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lab['value'] = pd.to_numeric(df_lab['value'], errors='coerce')
/home/jovyan/tmp/ipykernel_642872/1782168925.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_lab['value']=np.round(df_lab['value'],2)
/home/jovyan/tmp/ipykernel_642872/1782168925.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

 [2024-12-06 08:34:13][DEBUG]: == Working Agent: Basic Member Information Provider
 [2024-12-06 08:34:13][INFO]: == Starting Task: 
Retrieve and display the following information for the member:
1. Member ID: 15163
2. Age: 74.0
3. Gender: 0.0


Example :
Member Information

Member ID: 10
Age: 35
Gender: Male

Format the output like the top in markdown format.

Note: Display "Female" if the value of gender is 0, "Male" if the value of gender is 1, and "Unknown" for any other value.



> Entering new CrewAgentExecutor chain...


2024-12-06 08:34:15,004 - 139666044987136 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


I now can give a great answer  
Final Answer: 

```markdown
Member Information

Member ID: 15163
Age: 74
Gender: Female
```

> Finished chain.
 [2024-12-06 08:34:15][DEBUG]: == [Basic Member Information Provider] Task output: ```markdown
Member Information

Member ID: 15163
Age: 74
Gender: Female
```


 [2024-12-06 08:34:15][DEBUG]: == Working Agent: Past Medical Condition Analysis Agent
 [2024-12-06 08:34:15][INFO]: == Starting Task: Analyze the given             id         dos    type      code    value abnormal_cd                                                       cd_desc
344678   15163  2017-05-26    diag      E119      NaN         NaN                               TYPE 2 DM WITHOUT COMPLICATIONS
344679   15163  2017-05-26    diag       I10      NaN         NaN                                ESSENTIAL PRIMARY HYPERTENSION
344680   15163  2017-05-26    diag      R079      NaN         NaN                                        CHEST PAIN UNSPECIFIED
344681   15163  2017-07-07    d

Traceback (most recent call last):
  File "/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/httpx/_transports/default.py", line 72, in map_httpcore_exceptions
    yield
  File "/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/httpx/_transports/default.py", line 116, in __iter__
    for part in self._httpcore_stream:
  File "/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/httpcore/_sync/connection_pool.py", line 367, in __iter__
    raise exc from None
  File "/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/httpcore/_sync/connection_pool.py", line 363, in __iter__
    for part in self._stream:
  File "/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/httpcore/_sync/http11.py", line 349, in __iter__
    raise exc
  File "/home/jovyan/envs/llm_agents/lib/python3.11/site-packages/httpcore/_sync/http11.py", line 341, in __iter__
    for chunk in self._connection._receive_response_body(**kwargs):
  File "/home/jovyan/envs/llm_agents/lib/python3.1